# Example Notebook

This notebook provides an example in getting started with Stellar dataset. The notebook assumes that you have already familiarized yourself with the [README.md](https://github.com/stellar-gen-ai/stellar-dataset). 

The README.md contains instructions on how to download and process the dataset. After you have done so, you should place the dataset in a folder called `stellar-data` 

The folder structure should be

    .
    ├── ...
    ├── stellar-data                     # The datasets folder
    │   ├── 000                          # Zeroth subject folder
    │   │   ├── 0.jpg                    # First image file
    │   │   ├── 0_bg.png                 # First image mask file
    │   │   ├── 0_attributes.json        # First image celeba annotations
    │   │   ├── 1.jpg                    # Second image file
    │   │   ├── 1_bg.png                 # Second image mask file
    │   │   ├── 1_attributes.json        # Second image celeba annotations
    │   │   │ ...
    │   └── ...

In [ ]:
from stellar_dataset import Stellar

dataset = Stellar("stellar-data")

In [ ]:
sample = dataset[0]

In [ ]:
sample

In [ ]:
sample["img"]

## Metadata

The metadata can be used for the evaluation of an image

In [ ]:
dataset.get_metadata(sample["index"])